# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 03**: Data Cleaning and Transformation Pipeline

**Date**: September 18th 2025

**Student Name**: Antonia Hoerburger

**Professor**: Pablo Camarillo Ramirez

In [24]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when, concat_ws, trim, isnull
from antoniahorburger.spark_utils import SparkUtils

In [25]:
spark = SparkSession.builder \
    .appName("Examples on data sources (Files)") \
    .master("spark://aba0da925ac6:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [26]:
airlines_schema_columns = [("index", "int"), 
     ("airline", "string"), 
     ("flight", "string"),
     ("source_city", "string"),
     ("departure_time", "string"),
     ("stops", "string"),
     ("arrival_time", "string"),
     ("destination_city", "string"),
     ("class", "string"),
     ("duration", "float"),
     ("days_left", "int"),
     ("price", "int")
     ]
airlines_schema = SparkUtils.generate_schema(airlines_schema_columns)
airlines_schema

StructType([StructField('index', IntegerType(), True), StructField('airline', StringType(), True), StructField('flight', StringType(), True), StructField('source_city', StringType(), True), StructField('departure_time', StringType(), True), StructField('stops', StringType(), True), StructField('arrival_time', StringType(), True), StructField('destination_city', StringType(), True), StructField('class', StringType(), True), StructField('duration', FloatType(), True), StructField('days_left', IntegerType(), True), StructField('price', IntegerType(), True)])

In [27]:
df_airlines = spark.read \
                .option("header", "true") \
                .schema(airlines_schema) \
                .csv("./airlines_flights_data.csv")

df_airlines.show(n=5)

+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|    0|SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|
|    1|SpiceJet|SG-8157|      Delhi| Early_Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5953|
|    2| AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|
|    3| Vistara| UK-995|      Delhi|       Morning| zero|    Afternoon|          Mumbai|Economy|    2.25|        1| 5955|
|    4| Vistara| UK-963|      Delhi|       Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5955|
+-----+--------+-------+

In [28]:
print(f"before cleaning: {df_airlines.count()}")

df_airlines.select([count(when(isnull(c[0]) | col(c[0]).isNull(), c[0])).alias(c[0]) for c in airlines_schema_columns]).show()

airlines_clean = df_airlines \
        .dropDuplicates(["index"]) \
        .withColumn("airline", trim("airline")) \
        .withColumn("source_city", trim("source_city")) \
        .withColumn("destination_city", trim("destination_city")) \
        .filter(col("price").isNotNull())

airlines_clean_v2 = df_airlines.dropna()

print(f"after cleaning with trim: {airlines_clean.count()}")
airlines_clean.select([count(when(isnull(c[0]) | col(c[0]).isNull(), c[0])).alias(c[0]) for c in airlines_schema_columns]).show()

print(f"after cleaning with dropna: {airlines_clean_v2.count()}")
airlines_clean_v2.select([count(when(isnull(c[0]) | col(c[0]).isNull(), c[0])).alias(c[0]) for c in airlines_schema_columns]).show()

before cleaning: 300153


+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|index|airline|flight|source_city|departure_time|stops|arrival_time|destination_city|class|duration|days_left|price|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|    0|      0|     0|          0|             0|    0|           0|               0|    0|       0|        0|    0|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+



after cleaning with trim: 300153


+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|index|airline|flight|source_city|departure_time|stops|arrival_time|destination_city|class|duration|days_left|price|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|    0|      0|     0|          0|             0|    0|           0|               0|    0|       0|        0|    0|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+



after cleaning with dropna: 300153


[Stage 84:=============================>                            (1 + 1) / 2]

+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|index|airline|flight|source_city|departure_time|stops|arrival_time|destination_city|class|duration|days_left|price|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|    0|      0|     0|          0|             0|    0|           0|               0|    0|       0|        0|    0|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+



In [29]:
airlines_t1 = airlines_clean_v2.withColumn("stops_numeric",
                                           when(airlines_clean_v2.stops == "zero", lit(0))) \
                                .show(n=1)

+-----+--------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-----+-------------+
|index| airline| flight|source_city|departure_time|stops|arrival_time|destination_city|  class|duration|days_left|price|stops_numeric|
+-----+--------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-----+-------------+
|    0|SpiceJet|SG-8709|      Delhi|       Evening| zero|       Night|          Mumbai|Economy|    2.17|        1| 5953|            0|
+-----+--------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-----+-------------+
only showing top 1 row


In [30]:
airlines_clean = airlines_clean.withColumn(
    "stops_numeric",
    when(col("stops") == "zero", lit(0))
    .when(col("stops") == "one", lit(1))
    .when(col("stops") == "two", lit(2))
    .when(col("stops") == "three", lit(3))
    .otherwise(lit(None))
)

airlines_clean.select("stops", "stops_numeric").show(5)


+-----+-------------+
|stops|stops_numeric|
+-----+-------------+
| zero|            0|
| zero|            0|
| zero|            0|
| zero|            0|
| zero|            0|
+-----+-------------+
only showing top 5 rows


In [31]:
airlines_clean = airlines_clean.withColumn(
    "route",
    concat_ws(" → ", col("source_city"), col("destination_city"))
)

airlines_clean.select("source_city", "destination_city", "route").show(5)

[Stage 91:=============================>                            (1 + 1) / 2]

+-----------+----------------+--------------+
|source_city|destination_city|         route|
+-----------+----------------+--------------+
|      Delhi|          Mumbai|Delhi → Mumbai|
|      Delhi|          Mumbai|Delhi → Mumbai|
|      Delhi|          Mumbai|Delhi → Mumbai|
|      Delhi|          Mumbai|Delhi → Mumbai|
|      Delhi|          Mumbai|Delhi → Mumbai|
+-----------+----------------+--------------+
only showing top 5 rows


In [33]:
airlines_clean = airlines_clean.withColumn(
    "departure_time_num",
    when(col("departure_time") == "Early_Morning", lit(0))
    .when(col("departure_time") == "Morning", lit(1))
    .when(col("departure_time") == "Afternoon", lit(2))
    .when(col("departure_time") == "Evening", lit(3))
    .when(col("departure_time") == "Night", lit(4))
)

airlines_clean = airlines_clean.withColumn(
    "arrival_time_num",
    when(col("arrival_time") == "Early_Morning", lit(0))
    .when(col("arrival_time") == "Morning", lit(1))
    .when(col("arrival_time") == "Afternoon", lit(2))
    .when(col("arrival_time") == "Evening", lit(3))
    .when(col("arrival_time") == "Night", lit(4))
)

airlines_clean.select("departure_time", "departure_time_num", "arrival_time", "arrival_time_num").show(5)

[Stage 97:=============================>                            (1 + 1) / 2]

+--------------+------------------+------------+----------------+
|departure_time|departure_time_num|arrival_time|arrival_time_num|
+--------------+------------------+------------+----------------+
| Early_Morning|                 0|     Morning|               1|
|       Morning|                 1|   Afternoon|               2|
|       Morning|                 1|   Afternoon|               2|
|       Morning|                 1|     Morning|               1|
|     Afternoon|                 2|     Evening|               3|
+--------------+------------------+------------+----------------+
only showing top 5 rows


In [34]:
airlines_clean = airlines_clean.withColumn(
    "is_expensive",
    when(col("price") > 6000, lit(True)).otherwise(lit(False))
)

airlines_clean.select("price", "is_expensive").show(5)

[Stage 100:============================>                            (1 + 1) / 2]

+-----+------------+
|price|is_expensive|
+-----+------------+
| 5949|       false|
| 8790|        true|
| 9840|        true|
|11295|        true|
| 9945|        true|
+-----+------------+
only showing top 5 rows


In [35]:
airlines_clean.groupBy("airline").avg("price").show()

airlines_clean.groupBy("route").avg("duration").show()

airlines_clean.groupBy("airline").agg({"price":"min", "price":"max"}).show()

airlines_clean.groupBy("departure_time").count().show()

+---------+------------------+
|  airline|        avg(price)|
+---------+------------------+
|   Indigo| 5324.216303339517|
| SpiceJet| 6179.278881367218|
|Air_India| 23507.01911190229|
|  AirAsia|4091.0727419555224|
| GO_FIRST| 5652.007595045959|
|  Vistara| 30396.53630170735|
+---------+------------------+



+--------------------+------------------+
|               route|     avg(duration)|
+--------------------+------------------+
|Hyderabad → Banga...| 12.09331678643705|
|    Mumbai → Kolkata|12.836848115489666|
|    Mumbai → Chennai|12.665900287564627|
|  Mumbai → Hyderabad|13.263310412247066|
|  Mumbai → Bangalore|11.612022516178817|
|   Bangalore → Delhi|  9.77995566082195|
| Kolkata → Bangalore| 13.79294687524098|
|   Hyderabad → Delhi|10.829816602522587|
| Bangalore → Chennai|14.480207509137166|
|    Chennai → Mumbai|12.374656244132625|
|  Bangalore → Mumbai| 10.90507225639642|
| Chennai → Bangalore|13.952593563812163|
|      Mumbai → Delhi|  9.81805726844943|
|  Hyderabad → Mumbai|11.962923295795918|
|   Chennai → Kolkata|14.515774035955694|
|   Kolkata → Chennai|14.774181563782903|
| Kolkata → Hyderabad|13.853107514948396|
|   Delhi → Bangalore| 10.35412503844018|
|      Delhi → Mumbai|10.367774213738123|
| Hyderabad → Chennai|13.293238468912078|
+--------------------+------------

+---------+----------+
|  airline|max(price)|
+---------+----------+
|   Indigo|     31952|
| SpiceJet|     34158|
|Air_India|     90970|
|  AirAsia|     31917|
| GO_FIRST|     32803|
|  Vistara|    123071|
+---------+----------+



[Stage 121:============================>                            (1 + 1) / 2]

+--------------+-----+
|departure_time|count|
+--------------+-----+
|       Evening|65102|
|       Morning|71146|
|    Late_Night| 1306|
|     Afternoon|47794|
| Early_Morning|66790|
|         Night|48015|
+--------------+-----+



In [ ]:
sc.stop()